<a href="https://colab.research.google.com/github/Edward-Eughene-Timothy/Reinforcement-learning-with-Deep_Learning-for-Cartpole-Balancing/blob/main/Reinforcement_learning_with_Deep_Q_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Reinforcement learning with Deep-Q-Learning

### Dependancies :

In [ ]:
!pip install gym
!pip install keras-rl2
!pip install tensorflow==2.12.0
!pip install keras==2.12.0
import warnings
warnings.filterwarnings("ignore")



### Environment creation :

In [ ]:
import warnings
import gym as gy
import random as rm
warnings.filterwarnings("ignore")

In [ ]:
e=gy.make('CartPole-v0')
states=e.observation_space.shape[0] #available states in an environment
actions=e.action_space.n # number of action from a state

In [ ]:
states


4

In [ ]:
actions

2

In [ ]:
episodes = 10
for epi in range(1,episodes+1):
  state=e.reset()
  done=False
  score=0
  while not done :
    e.render()
    action = rm.choices([0, 1])[0]
    n_state, reward, done, info = e.step(action)
    score+=reward
  print('Episode:{} Score:{}'.format(epi, score))


Episode:1 Score:24.0
Episode:2 Score:15.0
Episode:3 Score:20.0
Episode:4 Score:15.0
Episode:5 Score:23.0
Episode:6 Score:20.0
Episode:7 Score:15.0
Episode:8 Score:47.0
Episode:9 Score:47.0
Episode:10 Score:38.0


### Deep learning model with Keras :

In [ ]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Flatten
from tensorflow.keras.optimizers.legacy import Adam


In [ ]:
def build_model(states,actions):
  model=Sequential()
  model.add(Flatten(input_shape=(1,states)))
  model.add(Dense(24,activation='relu'))
  model.add(Dense(24,activation='relu'))
  model.add(Dense(actions,activation='linear'))
  return model

In [ ]:
model=build_model(states,actions)
model.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_1 (Flatten)         (None, 4)                 0         
                                                                 
 dense_4 (Dense)             (None, 24)                120       
                                                                 
 dense_5 (Dense)             (None, 24)                600       
                                                                 
 dense_6 (Dense)             (None, 2)                 50        
                                                                 
Total params: 770
Trainable params: 770
Non-trainable params: 0
_________________________________________________________________


### Agent Building :

In [ ]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

In [ ]:
def build_agent(model,actions):
  policy=BoltzmannQPolicy()
  memory=SequentialMemory(limit=50000,window_length=1)
  dqn=DQNAgent(model=model,memory=memory,policy=policy,
               nb_actions=actions,nb_steps_warmup=10,target_model_update=1e-2)
  return dqn

In [ ]:
dqn=build_agent(model,actions)
dqn.compile(Adam(learning_rate=1e-3), metrics=['mae']) # Change Learning_rate to learning_rate
dqn.fit(e,nb_steps=50000,visualize=False,verbose=1)

Training for 50000 steps ...
Interval 1 (0 steps performed)
10000/10000 [==============================] - 85s 8ms/step - reward: 1.0000
84 episodes - episode_reward: 116.893 [9.000, 200.000] - loss: 4.565 - mae: 20.342 - mean_q: 41.065

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 84s 8ms/step - reward: 1.0000
51 episodes - episode_reward: 199.196 [172.000, 200.000] - loss: 10.114 - mae: 43.048 - mean_q: 86.704

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 85s 9ms/step - reward: 1.0000
50 episodes - episode_reward: 199.920 [196.000, 200.000] - loss: 9.485 - mae: 47.889 - mean_q: 96.334

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 90s 9ms/step - reward: 1.0000
50 episodes - episode_reward: 200.000 [200.000, 200.000] - loss: 8.850 - mae: 46.297 - mean_q: 93.021

Interval 5 (40000 steps performed)
10000/10000 [==============================] - 95s 9ms/step - reward: 1.0000
done

In [ ]:
scores=dqn.test(e,nb_episodes=100,visualize=False)
print(np.mean(scores.history['episode_reward']))

Testing for 100 episodes ...
Episode 1: reward: 200.000, steps: 200
Episode 2: reward: 200.000, steps: 200
Episode 3: reward: 200.000, steps: 200
Episode 4: reward: 200.000, steps: 200
Episode 5: reward: 200.000, steps: 200
Episode 6: reward: 200.000, steps: 200
Episode 7: reward: 200.000, steps: 200
Episode 8: reward: 200.000, steps: 200
Episode 9: reward: 200.000, steps: 200
Episode 10: reward: 200.000, steps: 200
Episode 11: reward: 200.000, steps: 200
Episode 12: reward: 200.000, steps: 200
Episode 13: reward: 200.000, steps: 200
Episode 14: reward: 200.000, steps: 200
Episode 15: reward: 200.000, steps: 200
Episode 16: reward: 200.000, steps: 200
Episode 17: reward: 200.000, steps: 200
Episode 18: reward: 200.000, steps: 200
Episode 19: reward: 200.000, steps: 200
Episode 20: reward: 200.000, steps: 200
Episode 21: reward: 200.000, steps: 200
Episode 22: reward: 200.000, steps: 200
Episode 23: reward: 200.000, steps: 200
Episode 24: reward: 200.000, steps: 200
Episode 25: reward: 

### Reloading Model :

In [ ]:
dqn.save_weights('dqn_weights.h5f',overwrite=True)


#### Deleting model, env and agent

In [36]:
'''del model
del dqn
del e'''  #Uncomment it for deleting

'del model\ndel dqn\ndel e'

#### Loading the model procedure after deleting

In [ ]:
env=gy.make('CartPole-v0')
actions=env.action_space.n
states=env.observation_space.shape[0]
model=build_model(states,actions)
dqn=build_agent(model,actions)
dqn.compile(Adam(lr=1e-3),metrics=['mae'])

In [ ]:
dqn.load_weights('dqn_weights.h5f')

In [ ]:
_=dqn.test(env,nb_episodes=5,visualize=True)

Testing for 5 episodes ...
Episode 1: reward: 200.000, steps: 200
Episode 2: reward: 200.000, steps: 200
Episode 3: reward: 200.000, steps: 200
Episode 4: reward: 200.000, steps: 200
Episode 5: reward: 200.000, steps: 200
